In [ ]:
# Cria features de lag e médias móveis
def create_lag_features(df, group_cols, value_col, lags=[1, 3, 6, 12]):
    """
    Cria features de lag e médias móveis
    """
    df_features = df.copy()
    
    # Ordena por grupo e data
    df_features = df_features.sort_values(group_cols + ['data'])
    
    # Cria lags
    for lag in lags:
        df_features[f'{value_col}_lag_{lag}'] = df_features.groupby(group_cols)[value_col].shift(lag)
    
    # Cria médias móveis
    for window in [3, 6, 12]:
        df_features[f'{value_col}_ma_{window}'] = df_features.groupby(group_cols)[value_col].rolling(window=window).mean().reset_index(0, drop=True)
    
    # Cria diferenças
    df_features[f'{value_col}_diff'] = df_features.groupby(group_cols)[value_col].diff()
    df_features[f'{value_col}_diff2'] = df_features.groupby(group_cols)[value_col].diff(2)
    
    # Cria features de tendência
    df_features[f'{value_col}_trend_3m'] = df_features.groupby(group_cols)[value_col].rolling(window=3).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0]).reset_index(0, drop=True)
    
    return df_features

# Aplica features de lag
df_crimes = create_lag_features(df_crimes, 
                               group_cols=['regiao_administrativa', 'tipo_crime'],
                               value_col='total_ocorrencias')

print("✅ Features de lag e médias móveis criadas")

# Mostra features de lag criadas
lag_features = [col for col in df_crimes.columns if 'lag_' in col or 'ma_' in col or 'diff' in col or 'trend' in col]
print(f"\n📋 Features de lag criadas: {len(lag_features)}")
for feature in lag_features[:10]:  # Mostra primeiras 10
    print(f"  - {feature}")


## 4. FEATURES ESPACIAIS
